-Each directory contains one CSV file with annotations ("GT-<ClassID>.csv") and the training images

-Each image is a traffic sign

-Image sizes varies 15x15 to 250x250

**Annotation Formation**

-Filename: Filename of corresponding image

-Width: Width of the image

-Height: Height of the image

-ROI.x1: X-coordinate of top-left corner of traffic sign bounding box

-ROI.y1: Y-coordinate of top-left corner of traffic sign bounding box

-ROI.x2: X-coordinate of bottom-right corner of traffic sign bounding box

-ROI.y2: Y-coordinate of bottom-right corner of traffic sign bounding box


In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#!pip install tensorflow_addons

In [3]:
import os
import pandas as pd
import random
import tensorflow as tf
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,Activation, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard, ReduceLROnPlateau
import pathlib
import tensorflow_addons as tfa
import sys
from PIL import Image
vcpi = sys.modules[__name__] 

c:\Users\Rengar\.conda\envs\vcpi2\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


Default Values

In [5]:
N_classes = 43
data = []
directory_training = "./datasets/train"
directory_test = "./datasets/test"
model_directory = "./models/"
ensemble_directory = "./ensemble/"
BATCH_SIZE = 32
IMAGE_SIZE = 32
N_CHANNELS = 3
KERNEL_SIZE = (3,3)
N_EPOCHS = 5
TRAIN_ONLINE = False
NUM_MODELS = 4

Filters
Ideas:
- Convert to RGB (grayscale I guess?)
- Translation
- Zooming
- Padding
- Change contrast 
- Use mean/median filters


In [6]:
N_FILTERS = 4
filters_name = ["rotate","brightness","translate","saturation"]

In [7]:
def filter_rotate(image,label):
    #Rotate image 10 degrees
    image = tfa.image.rotate(image, 10)

    return image, label

In [8]:
def filter_brightness(image,label):
    # Increase brightness of image
    image = tf.clip_by_value(tfa.image.random_hsv_in_yiq(image, 0.0, 1.0, 1.0, 0.1, 3.0),0,1)

    return image, label

In [9]:
def filter_shear(image, label):
    
    img = tfa.image.rotate(image, tf.random.uniform(shape=(), minval=-0.175, maxval=0.175))
    sx = tf.random.uniform(shape=(), minval=-0.1, maxval=0.1, dtype=tf.dtypes.float32)
    img = tfa.image.transform(img, [1, sx, -sx*32,   0,1,0,  0,0])
    return img, label

In [10]:
def filter_translate(image, label):

    img = tfa.image.rotate(image, tf.random.uniform(shape=(), minval=-0.175, maxval=0.175))
    tx = tf.random.uniform(shape=(), minval=-3, maxval=3, dtype=tf.dtypes.float32)
    ty = tf.random.uniform(shape=(), minval=-3, maxval=3, dtype=tf.dtypes.float32)  
    img = tfa.image.translate(img, [tx,ty])
    return img, label

In [11]:
def filter_saturation(image, label):
    
    img = tf.clip_by_value(tfa.image.random_hsv_in_yiq(image, 0.0, 1.0, 3.0, 1.0, 1.0),0,1)
    return img, label

In [12]:
filters = []

for filtro in filters_name:
    func = getattr(vcpi, "filter_{}".format(filtro))
    filters.append(func)

Models

In [13]:
def model_1_ensemble():
    modelLogits = Sequential()
    
    modelLogits.add(Conv2D(32, KERNEL_SIZE, input_shape=(IMAGE_SIZE, IMAGE_SIZE, N_CHANNELS)))         
    modelLogits.add(LeakyReLU(alpha=0.01))  
    modelLogits.add(BatchNormalization())
    modelLogits.add(Dropout(0.5)) 

    modelLogits.add(Conv2D(64, KERNEL_SIZE)) 
    modelLogits.add(LeakyReLU(alpha=0.01))
    modelLogits.add(MaxPooling2D(pool_size=(2, 2)))
    modelLogits.add(BatchNormalization())
    modelLogits.add(Dropout(0.5)) 

    modelLogits.add(Conv2D(128, KERNEL_SIZE))   
    modelLogits.add(LeakyReLU(alpha=0.01))
    modelLogits.add(MaxPooling2D(pool_size=(2, 2)))
    modelLogits.add(BatchNormalization())
    modelLogits.add(Dropout(0.5)) 
    
    modelLogits.add(Flatten())
    modelLogits.add(Dense(192))
    modelLogits.add(LeakyReLU(alpha=0.0))             
    modelLogits.add(Dropout(0.5)) 
    
    modelLogits.add(Dense(N_classes))
    
    output = Activation('softmax')(modelLogits.output)

    model = tf.keras.Model(modelLogits.inputs, output)
    
    opt = Adam(lr=0.0001)
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=[ 'accuracy'])
    
    return model, modelLogits

In [14]:
classNames = np.array(os.listdir(directory_training))
classNames

array(['00000', '00001', '00002', '00003', '00004', '00005', '00006',
       '00007', '00008', '00009', '00010', '00011', '00012', '00013',
       '00014', '00015', '00016', '00017', '00018', '00019', '00020',
       '00021', '00022', '00023', '00024', '00025', '00026', '00027',
       '00028', '00029', '00030', '00031', '00032', '00033', '00034',
       '00035', '00036', '00037', '00038', '00039', '00040', '00041',
       '00042'], dtype='<U5')

Saving images


In [15]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data = tf.data.Dataset.list_files(f"{directory_training}/*/*.png", shuffle=False)
image_count = len(data)
 

# Preparing datasets

## Aux functions

In [16]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == classNames

In [17]:
def get_img_label(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    
    img = tf.image.decode_png(img, channels=3)

    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [IMAGE_SIZE, IMAGE_SIZE])
    return img,label

In [18]:
dataset = data.map(get_img_label, num_parallel_calls=AUTOTUNE)
dataset_length = len(dataset)
print(dataset_length)

39209


Spliting dataset into training and validation sets, we using 80% and 20%.

In [19]:
val_size = int(image_count * 0.2)
train_ds = dataset.skip(val_size)
val_ds = dataset.take(val_size)

In [20]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
val_ds_size = tf.data.experimental.cardinality(val_ds).numpy()

print(train_ds_size)
print(val_ds_size)

31368
7841


Configuring dataset, to train a model it's important to have data to be well shuffled and to be batched

In [21]:

dataSolo = train_ds.cache()
# note: works the best if buffer size >= the full of size of the dataset
dataSolo = dataSolo.shuffle(buffer_size = train_ds_size)
# note2: we might need to tune or we just use autotune 
dataSolo = dataSolo.prefetch(buffer_size = train_ds_size)
dataSolo = dataSolo.batch(batch_size=BATCH_SIZE)
# note3: this allows later elements to be prepared while current element is being processed
# note4: it should end with a call to prefetch


In [22]:
val_set = val_ds.cache()
val_set = val_set.shuffle(buffer_size = val_ds_size)
val_set = val_set.prefetch(buffer_size = AUTOTUNE)
val_set = val_set.batch(batch_size=val_ds_size)

In [23]:
#i = 0
#test = next(iter(dataSolo))  
#
#print(test[0].shape, test[1].shape)

# TODO: FIX THIS IMAGES
# plt.figure(figsize=(10, 10))
# for i in range(25):
#   ax = plt.subplot(5, 5, i + 1)
#   plt.imshow(image_batch[i])
#   plt.axis("off")


Training Model

In [24]:
def prepare_callbacks(file_path):

    checkpointer = ModelCheckpoint(filepath= file_path, monitor = 'val_accuracy', verbose=1, save_weights_only=True, save_best_only=True)
    earlyStopper = EarlyStopping(monitor='val_accuracy', min_delta = 0.00001, patience = 15, verbose = 1)
    reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000000001, verbose = 1)

    return [checkpointer, earlyStopper, reduceLR]

In [25]:
file_path_prefix = f'{ensemble_directory}V1'

In [26]:
def train_models(train, val,file_path_prefix):

    models = []
    histories = []
    
    for i in range(NUM_MODELS):

        model, modelL = model_1_ensemble()

        if TRAIN_ONLINE:
            callbacks = prepare_callbacks(f'{file_path_prefix}_{i:02}/cp.ckpt')

            hist = model.fit(train, 
                            epochs = N_EPOCHS,
                            validation_data=val,
                            callbacks = callbacks)
            histories.append(hist)

        models.append([model, modelL])
    
    return models,histories

In [27]:
modelV1, historyV1 = train_models(dataSolo, val_set, file_path_prefix)

c:\Users\Rengar\.conda\envs\vcpi2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [28]:
def load_weights(models, file_path_prefix):

    for i in range(NUM_MODELS):

        file_path = f'{file_path_prefix}_{i:02}/cp.ckpt'

        models[i][0].load_weights(file_path)
        models[i][0].save('temp.hdf5')
        models[i][1].load_weights('temp.hdf5', by_name=True)
    

load_weights(modelV1, file_path_prefix)

In [29]:
test_set = tf.data.Dataset.list_files(f"{directory_test}/*/*.png")
test_set = test_set.map(get_img_label, num_parallel_calls=AUTOTUNE)
test_set_len = len(test_set)
data_test_set = test_set.batch(batch_size=BATCH_SIZE)

In [30]:
accuracy = 0

for i in range(NUM_MODELS):
    eval = modelV1[i][0].evaluate(data_test_set, verbose = 2)
    accuracy += eval[1]
    
print(f'average accuracy: {(accuracy/NUM_MODELS)*100:.3f}')   

395/395 - 5s - loss: 2.7189 - accuracy: 0.7455
395/395 - 3s - loss: 3.5412 - accuracy: 0.7674
395/395 - 3s - loss: 3.2371 - accuracy: 0.7560
395/395 - 3s - loss: 3.1986 - accuracy: 0.7626
average accuracy: 75.788


In [31]:
def get_labels_logits_and_preds(models):

    preds = [[] for _ in range(NUM_MODELS) ]
    logits = [[] for _ in range(NUM_MODELS)]
    labels = []

    for images, labs in data_test_set.take(-1):

        labels.extend(labs.numpy())
        for i in range(NUM_MODELS):
            preds[i].extend(models[i][0].predict(images))
            logits[i].extend(models[i][1].predict(images))

    labels = [np.argmax(i) for i in labels]  
    
    return labels, logits, preds

In [32]:
labels_V1, logits_V1, preds_V1 = get_labels_logits_and_preds(modelV1)

In [33]:
def get_class_preds(preds):

    class_preds = []

    for i in range(test_set_len):

        c = []
        for m in range(NUM_MODELS):

            c.append(np.argmax(preds[m][i]))
        class_preds.append(c)
        
    return class_preds



In [34]:
def get_class_from_sum_of_logits(logits):

    sum_logits = []

    for i in range(test_set_len):

        log = logits[0][i]
        for m in range(1, NUM_MODELS):
            log = np.add(log, logits[m][i])
        sum_logits.append(np.argmax(log))
    return(sum_logits)
    

In [35]:
class_preds_V1 = get_class_preds(preds_V1)
class_logits_V1 = get_class_from_sum_of_logits(logits_V1)    


In [36]:
import collections 
def get_stats(labels, class_preds, class_logits):

    all_correct = 0
    all_incorrect = 0
    maj_vote = 0
    maj_wrong = 0
    tie = 0
    count = 0
    log_ok = 0
    log_ko = 0

    for k in range(test_set_len):

        counter = collections.Counter(class_preds[k])
        if len(counter) == 1:
            if counter.most_common(1)[0][0] == labels[k]:
                all_correct += 1
            else:
                all_incorrect += 1
        else:
            aux = counter.most_common(2)
            if aux[0][1] > aux[1][1] and aux[0][0] == labels[k]:
                maj_vote += 1
            if aux[0][1] > aux[1][1] and aux[0][0] != labels[k]:
                maj_wrong += 1
            elif aux[0][1] == aux[1][1]:
                tie += 1
        if class_logits[k] == labels[k]:
            log_ok += 1
        else:
            log_ko += 1
        count += 1 
        
    return [count, all_correct, all_incorrect, maj_vote, tie, maj_wrong, log_ok, log_ko]
    
    
res = get_stats(labels_V1, class_preds_V1, class_logits_V1)
print(res, res[6]/res[0])

[12630, 9025, 1839, 532, 405, 829, 9699, 2931] 0.7679334916864609


In [37]:
dataV2 = dataset
dataV2 = dataV2.shuffle(buffer_size = len(dataV2))

for filtro in filters:
    dataV2 = dataV2.concatenate(dataV2.map(filtro))

image_count = len(dataV2)

dataV2 = dataV2.cache()
dataV2 = dataV2.batch(batch_size=BATCH_SIZE)
dataV2 = dataV2.prefetch(buffer_size=image_count)


In [38]:
file_path_prefix = f'{ensemble_directory}V3'

models_V2, histories_V2 = train_models(dataV2, val_set,file_path_prefix)

In [39]:
load_weights(models_V2, file_path_prefix)

In [40]:
accuracy = 0

for i in range(NUM_MODELS):
    eval = models_V2[i][0].evaluate(data_test_set, verbose = 2)
    accuracy += eval[1]
    
print(f'average accuracy: {(accuracy/NUM_MODELS)*100:.3f}')   

395/395 - 3s - loss: 0.0903 - accuracy: 0.9722
395/395 - 3s - loss: 0.0876 - accuracy: 0.9710
395/395 - 3s - loss: 0.0835 - accuracy: 0.9743
395/395 - 3s - loss: 0.0990 - accuracy: 0.9712
average accuracy: 97.219


In [41]:
labels_V2, logits_V2, preds_V2 = get_labels_logits_and_preds(models_V2)

In [42]:
class_preds_V2 = get_class_preds(preds_V2)
class_logits_V2 = get_class_from_sum_of_logits(logits_V2)    

In [43]:
res = get_stats(labels_V2, class_preds_V2, class_logits_V2)
print(res, res[6]/res[0])

[12630, 11979, 110, 319, 107, 115, 12362, 268] 0.9787806809184482


Stacked Ensemble

In [44]:
test_logits_preds = []

for i in range(len(test_set)):
    
    aux = []   
    for m in range(NUM_MODELS):        
        aux.extend(logits_V2[m][i])
        
    test_logits_preds.append(aux)
    

In [45]:
logits_train = [[] for _ in range(NUM_MODELS)]
labels_aux = []
for images, labs in dataV2.take(-1):

    labels_aux.extend(labs.numpy())
    for i in range(NUM_MODELS):
        logits_train[i].extend(models_V2[i][1].predict(images))
        
labels_train = [np.argmax(i) for i in labels_aux] 

print(logits_train[0])
for i in range(NUM_MODELS):
    logits_train[i][0]
## Build list of train inputs

In [ ]:
train_logits_preds = []

for i in range(image_count):
    
    aux = []
    
    for m in range(NUM_MODELS):
        
        aux.extend(logits_train[m][i])
        
    train_logits_preds.append(aux)

In [ ]:
print(len(labels_train), len(train_logits_preds))

Building Model

In [ ]:
stack_model  = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(len(train_logits_preds[0]),)),

  tf.keras.layers.Dense(192),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(64),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),

  tf.keras.layers.Dense(8, activation='softmax')
])


In [ ]:
stack_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
stack_model.fit(
    np.asarray(train_logits_preds),
    np.asarray(labels_train),
    epochs=5, 
    batch_size=32,
    validation_data = (np.asarray(test_logits_preds), np.asarray(labels_V2))
)

In [ ]:
pred = stack_model.predict(np.asarray(test_logits_preds))

In [ ]:
correct = 0

for i in range(test_set_len):
    if np.argmax(pred[i]) == labels_V2[i] :
        correct += 1
        
print(correct, correct/12630)